In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-03-27"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
12027,2024-03-27,República Tcheca Nbl,13:45,Kolin,Opava,3.86,1.23,170.5,1.83,1.87,0.0,0.0,0.0,4KLInbT6,0.259067,0.813008,0.546448,0.534759,0.072075,278.962,95.162305,0.341130,1.2,1.643168,1.369306,441.15,3.460,1.131923,0.327145,-52.0,181.532,145.600899,0.802067,1.2,1.643168,1.369306,118.80,2.184,1.900666,0.870268,29.0,85,99,5.19,1.20,224.410,211.868,0.730723,0.015289,NaN,-2.80,-0.560,-5.107143,0.000000,0.0,0.000000,-3.26,-0.652,-0.352761,0.482281,0.5,0.017719
12028,2024-03-27,República Tcheca Nbl,14:00,USK Praga,Decin,2.34,1.55,151.5,1.85,1.85,0.0,0.0,0.0,8hS9lKbf,0.427350,0.645161,0.540541,0.540541,0.072512,351.136,333.359308,0.949374,0.0,0.000000,NaN,936.00,5.102,4.517120,0.885363,-98.0,130.822,33.732890,0.257853,1.8,1.643168,0.912871,115.73,1.590,0.334888,0.210621,-12.0,72,71,13.00,1.63,195.736,230.528,0.287205,0.000000,NaN,-3.72,-0.744,-1.801075,0.606261,0.4,-0.206261,-0.69,-0.138,-3.985507,0.540538,0.5,-0.040538
12029,2024-03-27,República Tcheca Nbl,14:00,Usti n. Labem,Pardubice,1.32,3.20,164.5,1.82,1.88,0.0,0.0,0.0,QTMEmvE0,0.757576,0.312500,0.549451,0.531915,0.070076,243.288,124.482426,0.511667,1.8,1.643168,0.912871,426.96,3.052,1.794051,0.587828,21.0,203.846,85.417534,0.419030,1.8,1.643168,0.912871,242.78,2.688,1.228686,0.457101,19.0,72,61,5.93,3.98,153.066,211.090,0.588213,0.022933,NaN,3.80,0.760,0.421053,0.730264,0.9,0.169736,8.10,1.620,1.358025,0.414575,0.8,0.385425
12030,2024-03-27,República Tcheca Nbl,13:30,Ostrava,Srsni Pisek,1.37,2.93,168.5,1.85,1.85,0.0,0.0,0.0,SConSo4D,0.729927,0.341297,0.540541,0.540541,0.071224,137.542,56.733912,0.412484,0.6,1.341641,2.236068,200.10,1.720,0.457384,0.265921,-17.0,185.188,76.029253,0.410552,1.2,1.643168,1.369306,94.62,2.212,1.097939,0.496356,-7.0,87,83,2.30,1.14,115.078,357.776,0.513064,0.000000,NaN,-0.75,-0.150,-2.466667,0.625425,0.7,0.074575,-3.44,-0.688,-2.805233,0.256508,0.2,-0.056508
12031,2024-03-27,República Tcheca Nbl,14:30,Slavia Prague,Olomoucko,2.93,1.37,166.5,1.85,1.85,0.0,0.0,0.0,E9srTRl7,0.341297,0.729927,0.540541,0.540541,0.071224,287.714,124.901937,0.434118,1.2,1.643168,1.369306,447.04,3.758,1.439451,0.383037,-15.0,140.442,42.197692,0.300463,3.0,0.000000,0.000000,133.76,1.638,0.603921,0.368694,43.0,88,88,5.08,1.52,186.068,202.280,0.513064,0.000000,NaN,-5.00,-1.000,-1.930000,0.000000,0.0,0.000000,1.71,0.342,1.081871,0.360576,0.4,0.039424
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12152,2024-03-27,Europa Liga Bnxt,16:30,Spotter Leuven,Okapi Aalst,1.51,2.45,156.5,1.85,1.85,0.0,0.0,0.0,YHWgOH7O,0.662252,0.408163,0.540541,0.540541,0.070415,153.476,49.468848,0.322323,2.4,1.341641,0.559017,109.60,1.884,0.720437,0.382398,25.0,129.348,44.145916,0.341296,1.8,1.643168,0.912871,102.06,1.732,0.743317,0.429167,13.0,80,81,1.37,1.26,153.800,244.184,0.335697,0.000000,NaN,-0.72,-0.144,-3.541667,0.000000,0.0,0.000000,1.24,0.248,5.846774,0.000000,0.0,0.000000
12153,2024-03-27,Europa Liga Bnxt,16:30,Mechelen,Feyenoord,1.14,5.14,155.5,1.83,1.87,0.0,0.0,0.0,4xuwSa8a,0.877193,0.194553,0.546448,0.534759,0.071746,169.460,109.946226,0.648803,1.2,1.643168,1.369306,95.14,2.270,

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
12109,21:00,Eua Nba,Chicago Bulls,Indiana Pacers,2.27,Back Home
